# File input/output with scipy.io

* Loading and saving matlab filese

In [4]:
import numpy as np
from scipy import io as spio

a = np.ones((3,3))
spio.savemat('file.mat', {'a':a}) ## savemat expects a dictionary

data = spio.loadmat('file.mat', struct_as_record=True)
data['a']

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.]])

* Reading from tab-delimited files

In [16]:
import numpy as np
data = np.genfromtxt('input.tsv', dtype=['S3', 'i8', 'i8', 'S10'],
                    delimiter='\t', names=True)
data

array([(b'HSV', 21, 11, b'My favorit'), (b'FCB', 20, 10, b'Well, o.k.'),
       (b'BVB', 19, 11, b"It's cool!")], 
      dtype=[('Team', 'S3'), ('Score', '<i8'), ('Goals', '<i8'), ('Note', 'S10')])